# GraphDynS

This notebook reproduces the salient characteristics of the [GraphDynS](https://dl.acm.org/doi/10.1145/3352460.3358318) accelerator.

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from copy import deepcopy

from src import utils

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below. For BFS, use `interval=1`, for SSSP use an `interval=10` when constructing `G_SD`.

In [ ]:
V = 15

density = [1, 0.3]
seed = 0

N = 12
n = 2
m = 2
mask_shape = N // 2

G_SD = Tensor.fromRandom(rank_ids=["S", "D"], shape=[V, V], seed=seed, density=density, interval=1, name="G")

Visualize the initial graph's adjacency matrix. Skip this step if the graph is large (more than 70 edges).

In [ ]:
displayTensor(G_SD)

## Running GraphDynS with TeAAL/HiFiber

The TeAAL compiler currently only supports basic tensor algebra operations. Specifically, it has three restrictions:
- The **map** operator can only be either multiplication (`*`) or addition (`+`)
- The **reduce** operator can only be addition (`+`)
- The **default** for all outputs can only be `0`

However, graph algorithms require much more variety across all attributes. Therefore, in this notebook, we present a TeAAL specification that *must* be modified to accurately perform breadth-first search (BFS) or single-source shortest path (SSSP).

In this notebook, we do not differentiate between compressed and uncompressed iteration (all iteration is compressed).

### GraphDynS TeAAL Specification

For BFS/SSSP, the property we are trying to assign each vertex is its distance away from the start vertex. For a single iteration of the BFS/SSSP algorithm, we have the following tensors:
- `G`: Graph
- `A0`: Set of active vertices at the start of this iteration (initial frontier)
- `SO`: Graph edges whose sources are in `A0`
- `R`: Potential update to the vertex properties
- `MP`: Set of vertices that were updated on this iteration, paired with their initial property value
- `NP`: Set of vertices to be updated with the best property value
- `P0`: Set of vertex properties at the start of the iteration
- `P1`: Updated set of vertex properties
- `M`: Set of vertices that were actually updated on this iteration
- `A1`: Set of active vertices at the end of this iteration (updated frontier)

The TeAAL specification for GraphDynS is:
```yaml
einsum:
    declaration:
        G: [D, S]
        A0: [S]
        SO: [D, S]
        R: [V]
        MP: [V]
        NP: [V]
        P0: [V]
        P1: [V]
        M: [V]
        A1: [V]
    expressions:
        - SO[d, s] = take(G[d, s], A0[s], 0)
        - R[d] = SO[d, s] * A0[s]
        - MP[v] = take(R[v], P0[v], 1)
        - NP[v] = R[v] + MP[v]
        - M[v] = MP[v] + s * NP[v]
        - P1[v] = take(M[v], NP[v], 1)
        - A1[v] = take(M[v], NP[v], 1)
mapping:
    rank-order:
        G: [S, D]
    partitioning:
        SO:
            D: [uniform_shape(N), uniform_shape(mask_shape)]
            S: [uniform_shape(N), uniform_shape(m)]
        R:
            D: [uniform_shape(N), uniform_shape(mask_shape), uniform_shape(n)]
            S: [uniform_shape(N)]
            V: [follow(D)]
        MP:
            V: [uniform_shape(N), uniform_shape(mask_shape), uniform_shape(n)]
        NP:
            V: [uniform_shape(N), uniform_shape(mask_shape), uniform_shape(n)]
        M:
            V: [uniform_shape(N), uniform_shape(mask_shape), uniform_shape(n)]
        P1:
            V: [uniform_shape(N), uniform_shape(mask_shape), uniform_shape(n)]
        A1:
            V: [uniform_shape(N), uniform_shape(mask_shape), uniform_shape(n)]
    loop-order:
        SO: [D2, S2, S1, S0, D1, D0]
        R: [V3, S1, S0, V2, V1, V0]
        MP: [V3, V2, V1, V0]
        NP: [V3, V2, V1, V0]
        M: [V3, V2, V1, V0]
        P1: [V3, V2, V1, V0]
        A1: [V3, V2, V1, V0]
    spacetime:
        SO:
            space: [S0.coord]
            time: [D1, S2, S1, D0]
            opt: slip
        R:
            space: [D0.coord]
            time: [D2, S1, S0, D1]
            opt: slip
        MP:
            space: [V0.coord]
            time: [V3, V2, V1]
            opt: slip
        NP:
            space: [V0.coord]
            time: [V3, V2, V1]
            opt: slip
        M:
            space: [V0.coord]
            time: [V3, V2, V1]
            opt: slip
        P1:
            space: [V0.coord]
            time: [V3, V2, V1]
            opt: slip
        A1:
            space: [V0.coord]
            time: [V3, V2, V1]
            opt: slip
```

### Per-Run Initialization

Create the tensor of active vertices and the tensor of property values. The `start_vertex` can be modified below.

In [ ]:
start_vertex = 0

A0_S = Tensor.fromFiber(rank_ids=["S"], fiber=Fiber([start_vertex], [0]), default=float("inf"), name="A0")
P0_V = Tensor.fromFiber(rank_ids=["V"], fiber=Fiber([start_vertex], [0], default=float("inf")), default=float("inf"), name="P0")

### Modified HiFiber Ouput

Below is the HiFiber output obtained by compiling the above TeAAL specification (minus the `spacetime` stamp). It has been modified to actually perform BFS/SSSP.

In general, the changes are as follows:
- Multiplication (`*`) is remapped to addition (`+`)
- Addition (`+`) is remapped to minimum (`min`)
- Subtraction (`-` or `+ s *`) is remapped to not equal (`!=`)

To match this change, the default values for the following tensors need to be manually updated:
- `R`: `float("inf")`
- `M`: `False`
- `A1`: `float("inf")`

Additionally, Einsum notation is static single assigment (SSA), but the vector of properties is *updated* from iteration to iteration. So, instead of creating a new tensor `P1`, we must copy `P0` into `P1`.

Finally, we add a loop, which continues as long as there are active vertices.

The inline comments describe the changes.

In [ ]:
# The TeAAL specification is for a single iteration, to perform the full BFS/SSSP, loop until the set of active vertices is empty
while len(A0_S.getRoot()) > 0:

    # Track progress by displaying vertex distance seen so far
    P0_V.setName("P")
    displayTensor(P0_V)

    # Modified HiFiber
    SO_D2S2S1S0D1D0 = Tensor(rank_ids=["D2", "S2", "S1", "S0", "D1", "D0"], name="SO")
    tmp0 = G_SD
    tmp1 = tmp0.splitUniform(N, depth=1)
    tmp2 = tmp1.splitUniform(mask_shape, depth=2)
    G_SD2D1D0 = tmp2
    G_SD2D1D0.setRankIds(rank_ids=["S", "D2", "D1", "D0"])
    tmp3 = G_SD2D1D0
    tmp4 = tmp3.splitUniform(N, depth=0)
    tmp5 = tmp4.splitUniform(m, depth=1)
    G_S2S1S0D2D1D0 = tmp5
    G_S2S1S0D2D1D0.setRankIds(rank_ids=["S2", "S1", "S0", "D2", "D1", "D0"])
    tmp6 = A0_S
    tmp7 = tmp6.splitUniform(N, depth=0)
    tmp8 = tmp7.splitUniform(m, depth=1)
    A0_S2S1S0 = tmp8
    A0_S2S1S0.setRankIds(rank_ids=["S2", "S1", "S0"])
    so_d2 = SO_D2S2S1S0D1D0.getRoot()
    G_D2S2S1S0D1D0 = G_S2S1S0D2D1D0.swizzleRanks(rank_ids=["D2", "S2", "S1", "S0", "D1", "D0"])
    g_d2 = G_D2S2S1S0D1D0.getRoot()
    a0_s2 = A0_S2S1S0.getRoot()
    for d2, (so_s2, g_s2) in so_d2 << g_d2:
        for s2, (so_s1, (g_s1, a0_s1)) in so_s2 << (g_s2 & a0_s2):
            for s1, (so_s0, (g_s0, a0_s0)) in so_s1 << (g_s1 & a0_s1):
                for s0, (so_d1, (g_d1, a0_val)) in so_s0 << (g_s0 & a0_s0):
                    for d1, (so_d0, g_d0) in so_d1 << g_d1:
                        for d0, (so_ref, g_val) in so_d0 << g_d0:
                            so_ref <<= g_val
    tmp9 = SO_D2S2S1S0D1D0
    tmp10 = tmp9.swizzleRanks(rank_ids=["D2", "D1", "D0", "S2", "S1", "S0"])
    tmp11 = tmp10.mergeRanks(depth=3, levels=2, coord_style="absolute")
    tmp12 = tmp11.mergeRanks(depth=0, levels=2, coord_style="absolute")
    tmp12.setRankIds(rank_ids=["D", "S"])
    SO_DS = tmp12
    # Custom default
    R_V3V2V1V0 = Tensor(rank_ids=["V3", "V2", "V1", "V0"], default=float("inf"), name="R")
    tmp13 = SO_DS
    tmp14 = tmp13.splitUniform(N, depth=0)
    tmp15 = tmp14.splitUniform(mask_shape, depth=1)
    tmp16 = tmp15.splitUniform(n, depth=2)
    SO_D3D2D1D0S = tmp16
    SO_D3D2D1D0S.setRankIds(rank_ids=["D3", "D2", "D1", "D0", "S"])
    tmp17 = SO_D3D2D1D0S
    tmp18 = tmp17.splitUniform(N, depth=4)
    SO_D3D2D1D0S1S0 = tmp18
    SO_D3D2D1D0S1S0.setRankIds(rank_ids=["D3", "D2", "D1", "D0", "S1", "S0"])
    tmp19 = A0_S
    tmp20 = tmp19.splitUniform(N, depth=0)
    A0_S1S0 = tmp20
    A0_S1S0.setRankIds(rank_ids=["S1", "S0"])
    r_v3 = R_V3V2V1V0.getRoot()
    SO_D3S1S0D2D1D0 = SO_D3D2D1D0S1S0.swizzleRanks(rank_ids=["D3", "S1", "S0", "D2", "D1", "D0"])
    so_d3 = SO_D3S1S0D2D1D0.getRoot()
    a0_s1 = A0_S1S0.getRoot()
    for v3, (r_v2, so_s1) in r_v3 << so_d3.project(trans_fn=lambda d3: d3):
        for s1, (so_s0, a0_s0) in so_s1 & a0_s1:
            for s0, (so_d2, a0_val) in so_s0 & a0_s0:
                for v2, (r_v1, so_d1) in r_v2 << so_d2.project(trans_fn=lambda d2: d2):
                    inputs_v1 = Fiber.fromLazy(so_d1.project(trans_fn=lambda d1: d1))
                    for v1_pos, (v1, (r_v0, so_d0)) in enumerate(r_v1 << so_d1.project(trans_fn=lambda d1: d1)):
                        if v1_pos == 0:
                            v0_start = 0
                        else:
                            v0_start = v1
                        if v1_pos + 1 < len(inputs_v1):
                            v0_end = inputs_v1.getCoords()[v1_pos + 1]
                        else:
                            v0_end = V
                        for v0, (r_ref, so_val) in r_v0 << so_d0.project(trans_fn=lambda d0: d0, interval=(v0_start, v0_end)):
                            # + => min, * => +
                            r_ref <<= min(r_ref, so_val + a0_val)
    tmp21 = R_V3V2V1V0
    tmp22 = tmp21.mergeRanks(depth=0, levels=3, coord_style="absolute")
    tmp22.setRankIds(rank_ids=["V"])
    R_V = tmp22
    # Custom default
    # Explicit shape to deal with the fact that sometimes MP is empty
    MP_V3V2V1V0 = Tensor(rank_ids=["V3", "V2", "V1", "V0"], default=float("inf"), name="MP", shape=[V, V, V, V])
    tmp23 = R_V
    tmp24 = tmp23.splitUniform(N, depth=0)
    tmp25 = tmp24.splitUniform(mask_shape, depth=1)
    tmp26 = tmp25.splitUniform(n, depth=2)
    R_V3V2V1V0 = tmp26
    R_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    tmp27 = P0_V
    tmp28 = tmp27.splitUniform(N, depth=0)
    tmp29 = tmp28.splitUniform(mask_shape, depth=1)
    tmp30 = tmp29.splitUniform(n, depth=2)
    P0_V3V2V1V0 = tmp30
    P0_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    mp_v3 = MP_V3V2V1V0.getRoot()
    r_v3 = R_V3V2V1V0.getRoot()
    p0_v3 = P0_V3V2V1V0.getRoot()
    for v3, (mp_v2, (r_v2, p0_v2)) in mp_v3 << (r_v3 & p0_v3):
        for v2, (mp_v1, (r_v1, p0_v1)) in mp_v2 << (r_v2 & p0_v2):
            for v1, (mp_v0, (r_v0, p0_v0)) in mp_v1 << (r_v1 & p0_v1):
                for v0, (mp_ref, (r_val, p0_val)) in mp_v0 << (r_v0 & p0_v0):
                    mp_ref <<= p0_val
    tmp31 = MP_V3V2V1V0
    tmp32 = tmp31.mergeRanks(depth=0, levels=3, coord_style="absolute")
    tmp32.setRankIds(rank_ids=["V"])
    MP_V = tmp32
    # Custom default
    NP_V3V2V1V0 = Tensor(rank_ids=["V3", "V2", "V1", "V0"], default=float("inf"), name="NP")
    tmp33 = R_V
    tmp34 = tmp33.splitUniform(N, depth=0)
    tmp35 = tmp34.splitUniform(mask_shape, depth=1)
    tmp36 = tmp35.splitUniform(n, depth=2)
    R_V3V2V1V0 = tmp36
    R_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    tmp37 = MP_V
    tmp38 = tmp37.splitUniform(N, depth=0)
    tmp39 = tmp38.splitUniform(mask_shape, depth=1)
    tmp40 = tmp39.splitUniform(n, depth=2)
    MP_V3V2V1V0 = tmp40
    MP_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    np_v3 = NP_V3V2V1V0.getRoot()
    r_v3 = R_V3V2V1V0.getRoot()
    mp_v3 = MP_V3V2V1V0.getRoot()
    for v3, (np_v2, (_, r_v2, mp_v2)) in np_v3 << (r_v3 | mp_v3):
        for v2, (np_v1, (_, r_v1, mp_v1)) in np_v2 << (r_v2 | mp_v2):
            for v1, (np_v0, (_, r_v0, mp_v0)) in np_v1 << (r_v1 | mp_v1):
                for v0, (np_ref, (_, r_val, mp_val)) in np_v0 << (r_v0 | mp_v0):
                    # + => min
                    np_ref <<= min(r_val, mp_val)
    tmp41 = NP_V3V2V1V0
    tmp42 = tmp41.mergeRanks(depth=0, levels=3, coord_style="absolute")
    tmp42.setRankIds(rank_ids=["V"])
    NP_V = tmp42
    # Custom default
    M_V3V2V1V0 = Tensor(rank_ids=["V3", "V2", "V1", "V0"], default=False, name="M")
    tmp43 = MP_V
    tmp44 = tmp43.splitUniform(N, depth=0)
    tmp45 = tmp44.splitUniform(mask_shape, depth=1)
    tmp46 = tmp45.splitUniform(n, depth=2)
    MP_V3V2V1V0 = tmp46
    MP_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    tmp47 = NP_V
    tmp48 = tmp47.splitUniform(N, depth=0)
    tmp49 = tmp48.splitUniform(mask_shape, depth=1)
    tmp50 = tmp49.splitUniform(n, depth=2)
    NP_V3V2V1V0 = tmp50
    NP_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    m_v3 = M_V3V2V1V0.getRoot()
    mp_v3 = MP_V3V2V1V0.getRoot()
    np_v3 = NP_V3V2V1V0.getRoot()
    for v3, (m_v2, (_, mp_v2, np_v2)) in m_v3 << (mp_v3 | np_v3):
        for v2, (m_v1, (_, mp_v1, np_v1)) in m_v2 << (mp_v2 | np_v2):
            for v1, (m_v0, (_, mp_v0, np_v0)) in m_v1 << (mp_v1 | np_v1):
                for v0, (m_ref, (_, mp_val, np_val)) in m_v0 << (mp_v0 | np_v0):
                    # - => !=
                    m_ref <<= mp_val != np_val
    tmp51 = M_V3V2V1V0
    tmp52 = tmp51.mergeRanks(depth=0, levels=3, coord_style="absolute")
    tmp52.setRankIds(rank_ids=["V"])
    M_V = tmp52
    # P0 and P1 are actually the same tensor
    P1_V3V2V1V0 = deepcopy(P0_V3V2V1V0)
    tmp53 = M_V
    tmp54 = tmp53.splitUniform(N, depth=0)
    tmp55 = tmp54.splitUniform(mask_shape, depth=1)
    tmp56 = tmp55.splitUniform(n, depth=2)
    M_V3V2V1V0 = tmp56
    M_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    tmp57 = NP_V
    tmp58 = tmp57.splitUniform(N, depth=0)
    tmp59 = tmp58.splitUniform(mask_shape, depth=1)
    tmp60 = tmp59.splitUniform(n, depth=2)
    NP_V3V2V1V0 = tmp60
    NP_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    p1_v3 = P1_V3V2V1V0.getRoot()
    m_v3 = M_V3V2V1V0.getRoot()
    np_v3 = NP_V3V2V1V0.getRoot()
    for v3, (p1_v2, (m_v2, np_v2)) in p1_v3 << (m_v3 & np_v3):
        for v2, (p1_v1, (m_v1, np_v1)) in p1_v2 << (m_v2 & np_v2):
            for v1, (p1_v0, (m_v0, np_v0)) in p1_v1 << (m_v1 & np_v1):
                for v0, (p1_ref, (m_val, np_val)) in p1_v0 << (m_v0 & np_v0):
                    p1_ref <<= np_val
    tmp61 = P1_V3V2V1V0
    tmp62 = tmp61.mergeRanks(depth=0, levels=3, coord_style="absolute")
    tmp62.setRankIds(rank_ids=["V"])
    P1_V = tmp62
    # Custom default
    A1_V3V2V1V0 = Tensor(rank_ids=["V3", "V2", "V1", "V0"], default=float("inf"), name="A1")
    tmp63 = M_V
    tmp64 = tmp63.splitUniform(N, depth=0)
    tmp65 = tmp64.splitUniform(mask_shape, depth=1)
    tmp66 = tmp65.splitUniform(n, depth=2)
    M_V3V2V1V0 = tmp66
    M_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    tmp67 = NP_V
    tmp68 = tmp67.splitUniform(N, depth=0)
    tmp69 = tmp68.splitUniform(mask_shape, depth=1)
    tmp70 = tmp69.splitUniform(n, depth=2)
    NP_V3V2V1V0 = tmp70
    NP_V3V2V1V0.setRankIds(rank_ids=["V3", "V2", "V1", "V0"])
    a1_v3 = A1_V3V2V1V0.getRoot()
    m_v3 = M_V3V2V1V0.getRoot()
    np_v3 = NP_V3V2V1V0.getRoot()
    for v3, (a1_v2, (m_v2, np_v2)) in a1_v3 << (m_v3 & np_v3):
        for v2, (a1_v1, (m_v1, np_v1)) in a1_v2 << (m_v2 & np_v2):
            for v1, (a1_v0, (m_v0, np_v0)) in a1_v1 << (m_v1 & np_v1):
                for v0, (a1_ref, (m_val, np_val)) in a1_v0 << (m_v0 & np_v0):
                    a1_ref <<= np_val
    tmp71 = A1_V3V2V1V0
    tmp72 = tmp71.mergeRanks(depth=0, levels=3, coord_style="absolute")
    tmp72.setRankIds(rank_ids=["V"])
    A1_V = tmp72
    
    # Prepare for the next iteration
    P0_V = P1_V
    A0_S = A1_V

### Check Results

Check that generated code computes the correct result.

**Note**: Should be used after running the kernel (above cell).

In [ ]:
utils.check_bfs_sssp(G_SD, start_vertex, P1_V)